In [17]:
import pandas as pd
from sklearn.model_selection import train_test_split
import warnings
warnings.simplefilter('ignore')

In [18]:
data = pd.read_csv("C:\\Users\\athar\\OneDrive\\Documents\\vsc-files\\3rd yr\\developer mode\\projects\\phishing-domain-detection\\dataset\\dataset_full.csv")
data.head()

,qty_dot_url,qty_hyphen_url,qty_underline_url,qty_slash_url,qty_questionmark_url,qty_equal_url,qty_at_url,qty_and_url,qty_exclamation_url,qty_space_url,...,qty_ip_resolved,qty_nameservers,qty_mx_servers,ttl_hostname,tls_ssl_certificate,qty_redirects,url_google_index,domain_google_index,url_shortened,phishing
0,3,0,0,1,0,0,0,0,0,0,...,1,2,0,892,0,0,0,0,0,1
1,5,0,1,3,0,3,0,2,0,0,...,1,2,1,9540,1,0,0,0,0,1
2,2,0,0,1,0,0,0,0,0,0,...,1,2,3,589,1,0,0,0,0,0
3,4,0,2,5,0,0,0,0,0,0,...,1,2,0,292,1,0,0,0,0,1
4,2,0,0,0,0,0,0,0,0,0,...,1,2,1,3597,0,1,0,0,0,0


In [19]:
num_data_col = []
for i in data.columns:
    if data[i].nunique() > 20:
        num_data_col.append(i)

In [20]:
cat_data_col = []
for i in data.columns:
    if data[i].nunique() < 20:
        cat_data_col.append(i)

In [21]:
cat_data_col.remove('phishing')

In [22]:
x = data.drop(columns = "phishing", axis = 1)
y = data['phishing']

In [23]:
xtrain,xtest,ytrain,ytest = train_test_split(x,y,test_size=0.33, random_state=42)

In [24]:
from sklearn.preprocessing import RobustScaler as RS
rs = RS()
rs1 = rs.fit(xtrain)
xtrain = rs1.transform(xtrain)
xtest = rs1.transform(xtest)

In [25]:
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score

from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import SGDClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from xgboost import XGBClassifier


import pickle
from datetime import datetime

In [26]:
lr = LogisticRegression()
nb = GaussianNB()
sgd = SGDClassifier()
knn = KNeighborsClassifier(n_neighbors=15)
# dtree = DecisionTreeClassifier()
rf = RandomForestClassifier(oob_score=True,max_features=None)
svc = SVC(kernel="linear")
xgb = XGBClassifier()

models = [lr,nb,sgd,knn#,dtree,rf
          ,xgb,svc]
model_names = ["Logistic Regression", "Naive Bayes", "Stochastic Gradient Descent", "K Nearest Neighbors"#, "decision Tree", "Random Forest"
               , "XGBoost Classifier","Support Vector Machine Classifier"]
model_pred = []
model_score = []
model_precession = []
model_recall = []
model_f1 = []
# model_roc_auc = []
i=0
for model in models:
    print(f"Currently running {model_names[i]}")
    i+=1
    model.fit(xtrain,ytrain)
    model_pred.append(model.predict(xtest))
    model_score.append(model.score(xtest,ytest))
    model_precession.append(precision_score(y_true = ytest, y_pred = model_pred[-1]))
    model_recall.append(recall_score(ytest,model_pred[-1]))
    model_f1.append(f1_score(ytest,model_pred[-1]))
    filename = f'models/{model_names[i]}.sav'
    pickle.dump(model, open(filename, 'wb'))
#     model_roc_auc.append(roc_auc_score(ytest, model.predict_proba(xtest)[:, 1]))
    print("\t score",model_score[-1])
    print("\t precision",model_precession[-1])
    print("\t recall",model_recall[-1])
    print("\t F1 score",model_f1[-1])
    print("----------------------------------------------------")

Currently running Logistic Regression
	 score 0.9312230806043618
	 precision 0.8906976744186047
	 recall 0.9122667725287813
	 F1 score 0.9013532065110806
----------------------------------------------------
Currently running Naive Bayes
	 score 0.8647022629384016
	 precision 0.8708777885548011
	 recall 0.712882096069869
	 F1 score 0.7839991268282034
----------------------------------------------------
Currently running Stochastic Gradient Descent
	 score 0.9133110002051001
	 precision 0.917312375470445
	 recall 0.8224493846764589
	 F1 score 0.8672946101517531
----------------------------------------------------
Currently running K Nearest Neighbors
	 score 0.9428112394886169
	 precision 0.9113874473709977
	 recall 0.9237792774910679
	 F1 score 0.9175415249642664
----------------------------------------------------
Currently running XGBoost Classifier
	 score 0.9693033431325631
	 precision 0.9536378015025702
	 recall 0.9574235807860262
	 F1 score 0.9555269413629159
---------------------

IndexError: list index out of range